In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
pd.set_option('max_columns',105)
%matplotlib inline
sns.set()

from scipy import stats
from scipy.stats import skew
from math import sqrt

#plotly
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

#sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler,RobustScaler,QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,HuberRegressor,Lasso,ElasticNet,BayesianRidge

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor


import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from mlxtend.regressor import StackingRegressor

In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
#warnings.filterwarnings('ignore')

from subprocess import check_output

input = Path('input')
#print(check_output(['ls',input]).decode('utf-8'))   エラーが出る、後で検討

NameError: name 'Path' is not defined

In [ ]:
def get_best_score(grid):
    
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)
    print(grid.best_params_)
    print(grid.best_estimator_)
    
    return best_score

In [ ]:
def plotly_scatter_x_y(df_plot,val_x,val_y):
    
    value_x = df_plot[val_x]
    value_y = df_plot[val_y]
    
    trace_1 = go.Scatter(x=value_x,y=value_y,name=val_x,mode='markers',opacity=0.8)
    
    data = [trace_1]
    
    plot_title = val_y + ' vs. ' + val_x
    
    layout = dict(title = plot_title,
                 xaxis=dict(title=val_x,ticklen=5,zeroline=False),
                 yaxis=dict(title=val_y,side='left'),
                 legend=dict(orientation='h',x=0.4,y=1.0),
                 autosize=False,width=750,height=500)
    fig = dict(data=data,layout=layout)
    iplot(fig)

In [19]:
def plotly_scatter_x_y_color(df_plot,val_x,val_y,val_z):
    
    value_x = df_plot[val_x]
    value_y = df_plot[val_y]
    value_z = df_plot[val_z]
    
    trace_1 = go.Scatter(x=value_x,y=value_y,name=val_x,
                        mode='markers',opacity=0.8,text=value_z,
                        marker=dict(size=6,color=value_z,
                                   colorscale='Jet',showscale=True))
    
    data = [trace_1]
    
    plot_title = val_y + ' vs. ' + val_x
    
    layout = dict(title=plot_title,
                 xaxis=dict(title=val_x,ticklen=5,zeroline=False),
                 yaxis=dict(title=val_y,side='left'),
                 legend=dict(orientation='h',x=0.4,y=1.0),
                 autosize=False,width=750,height=500)
    
    fig = dict(data=data, layout=layout)
    iplot(fit)

In [96]:
def plotly_scatter_x_y_catg_color(df,val_x,val_y,val_z):
    
    catg_for_colors = sorted(df[val_z].unique().tolist())

    fig = {'data':[{'x': df[df[val_z]==catg][val_x],
                    'y': df[df[val_z]==catg][val_y],
                    'name':catg,
                    'mode': 'markers',
                    'marker':{'size':6}
                    } for catg in catg_for_colors ],
            'layout': {'xaxis':{'title':val_x},
                        'yaxis':{'title':val_y},
                        'colorway':['#a9a9a9','#e6beff','#911eb4','#4363d8',
                                   '#42d4f4','#3cb44b','#bfef45','#ffe119','#f58231',
                                   '#e6194B'],
                        'autosize':False,
                        'width':750,
                        'height':600}}
    iplot(fig)

In [103]:
def plotly_scatter3d(data,feat1,feat2,target):
    
    df = data
    x = df[feat1]
    y = df[feat2]
    z = df[target]
    
    trace1 = go.Scatter3d(x=x,y=y,z=z,
                         mode='markers',
                         marker=dict(size=5,color=y,
                                    colorscale='Viridis',
                                    opacity=0.8))
    data = [trace1]
    camera = dict(up=dict(x=0,y=0,z=1),
                 center=dict(x=0,y=0,z=0.0),
                 eye=dict(x=2.5,y=0.1,z=0.8))
    
    layout = go.Layout(title = target + 'as function of ' + feat1 + ' and ' + feat2,
                      autosize=False,width=700,height=600,
                      margin=dict(l=15,r=25,b=15,t=30),
                      scene=dict(camera=camera,
                                xaxis=dict(title=feat1),
                                yaxis=dict(title=feat2),
                                zaxis=dict(title=target)
                                )
                      )
    
    fig = go.Figure(data=data,layout=layout)
    iplot(fig)

In [23]:
df_train = pd.read_csv(input / 'train.csv')
df_test = pd.read_csv(input / 'test.csv')

In [24]:
print('df_train.shape:',df_train.shape)
print('*'*45)
print('df_test.shape:',df_test.shape)

df_train.shape: (1200, 81)
*********************************************
df_test.shape: (800, 80)


In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            1200 non-null   int64 
 1   Order            1200 non-null   int64 
 2   PID              1200 non-null   int64 
 3   MS SubClass      1200 non-null   int64 
 4   MS Zoning        1200 non-null   object
 5   Lot Frontage     1200 non-null   int64 
 6   Lot Area         1200 non-null   int64 
 7   Street           1200 non-null   object
 8   Alley            1200 non-null   object
 9   Lot Shape        1200 non-null   object
 10  Land Contour     1200 non-null   object
 11  Utilities        1200 non-null   object
 12  Lot Config       1200 non-null   object
 13  Land Slope       1200 non-null   object
 14  Neighborhood     1200 non-null   object
 15  Condition 1      1200 non-null   object
 16  Condition 2      1200 non-null   object
 17  Bldg Type        1200 non-null   

In [26]:
df_train.head()

,index,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,0,2827,911587246,18,RL,68,9241,Pave,Grvl,IR2,Lvl,AllPub,Corner,Gtl,Mitchel,Norm,Norm,1Fam,2Story,6,4,1999,2010,Gable,CompShg,AsbShng,VinylSd,BrkFace,19,TA,TA,PConc,TA,TA,Av,GLQ,30,Unf,0,1343,GasA,Ex,N,SBrkr,523,551,0,1283,0,2,0,3,1,TA,5,Typ,0,Gd,Attchd,1979,Unf,3,534,TA,TA,Y,253,38,0,0,0,0,Gd,MnPrv,Shed,0,2,2005,WD,Normal,94946
1,3,1239,505139030,22,RL,50,6249,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1.5Fin,7,5,1941,2005,Gable,CompShg,VinylSd,MetalSd,None,17,Gd,TA,CBlock,Gd,TA,Av,Rec,528,Unf,0,1202,GasA,TA,Y,SBrkr,815,39,1,1075,0,2,0,3,2,Gd,4,Typ,0,Gd,Attchd,1951,Unf,2,188,TA,TA,P,14,0,0,1,0,0,Gd,MnPrv,Shed,0,5,2009,WD,Partial,162272
2,7,2768,524499845,64,RL,55,8757,Pave,Grvl,Reg,Lvl,AllPub,CulDSac,Gtl,MeadowV,Norm,Norm,1Fam,2Story,5,6,1980,2004,Gable,CompShg,VinylSd,MetalSd,None,16,Gd,TA,BrkTil,TA,TA,Mn,ALQ,1866,Unf,0,825,GasA,TA,Y,FuseA,786,0,2,1210,0,1,0,3,0,Gd,7,Typ,0,Gd,Attchd,1977,RFn,2,300,TA,TA,Y,343,0,5,0,0,0,Gd,MnPrv,Shed,0,7,2009,WD,Normal,129620
3,10,2120,890941778,60,RL,43,12946,Grvl,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1.5Fin,6,4,2009,1954,Hip,CompShg,VinylSd,HdBoard,Stone,340,Gd,TA,PConc,TA,TA,Gd,Unf,65,Unf,650,787,GasA,TA,Y,SBrkr,1446,0,3,1203,0,1,1,3,1,Gd,8,Typ,0,Gd,Attchd,1976,Unf,1,706,TA,TA,Y,11,0,2,0,0,0,Gd,MnPrv,Shed,0,6,2007,WD,Normal,124625
4,11,407,902420295,64,RM,137,5902,Pave,Grvl,Reg,Low,AllPub,Inside,Gtl,Edwards,RRAe,Norm,1Fam,1Story,8,4,2011,1964,Gable,CompShg,VinylSd,CmentBd,None,12,TA,Gd,BrkTil,TA,TA,Mn,BLQ,0,BLQ,97,713,GasA,Ex,Y,FuseA,563,381,2,939,0,0,0,3,1,Gd,6,Typ,2,TA,Attchd,1956,Fin,2,447,TA,TA,Y,4,0,4,0,0,2,Gd,MnPrv,Shed,0,5,2007,WD,Normal,145879


In [27]:
df_train.drop('index',axis=1,inplace=True)
id_test = df_test['index']
df_test.drop('index',axis=1,inplace=True)

In [28]:
df_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Order,1200.0,1.838832e+03,9.070042e+02,0.0,1.040000e+03,2070.5,2.605250e+03,3378.0
PID,1200.0,8.017210e+08,1.620446e+08,495286289.0,8.674117e+08,888073650.5,8.994243e+08,939019144.0
MS SubClass,1200.0,6.983333e+01,4.471070e+01,15.0,4.275000e+01,61.0,8.825000e+01,225.0
Lot Frontage,1200.0,6.787083e+01,1.953357e+01,0.0,5.775000e+01,68.0,7.800000e+01,185.0
Lot Area,1200.0,1.060305e+04,6.066948e+03,0.0,7.580250e+03,9795.5,1.218325e+04,81991.0
Overall Qual,1200.0,6.045833e+00,1.486239e+00,1.0,5.000000e+00,6.0,7.000000e+00,11.0
Overall Cond,1200.0,4.562500e+00,1.011738e+00,0.0,4.000000e+00,4.0,5.000000e+00,9.0
Year Built,1200.0,1.969611e+03,2.911857e+01,1871.0,1.952000e+03,1971.0,1.994000e+03,2018.0
Year Remod/Add,1200.0,1.990041e+03,2.034705e+01,1944.0,1.975000e+03,1997.0,2.008000e+03,2014.0
Mas Vnr Area,1200.0,9.076333e+01,1.787684e+02,0.0,9.000000e+00,15.0,2.200000e+01,1121.0


In [31]:
#カテゴリカルデータの統計表
df_train.describe(include=['O']).transpose()

,count,unique,top,freq
MS Zoning,1200,6,RL,974
Street,1200,2,Pave,1195
Alley,1200,2,Grvl,1159
Lot Shape,1200,4,Reg,925
Land Contour,1200,4,Lvl,962
Utilities,1200,2,AllPub,1189
Lot Config,1200,5,Inside,929
Land Slope,1200,3,Gtl,1145
Neighborhood,1200,28,OldTown,127
Condition 1,1200,9,Norm,1055


In [32]:
df_train_null = pd.DataFrame()
df_train_null['missing'] = df_train.isnull().sum()[df_train.isnull().sum()>0].sort_values(ascending=False)

df_test_null = pd.DataFrame(df_test.isnull().sum(),columns=['missing'])
df_test_null = df_test_null.loc[df_test_null['missing']>0]

In [35]:
trace1 = go.Bar(x = df_train_null.index,y = df_train_null['missing'],
               name='df_train',text=df_train_null.index)

trace2 = go.Bar(x = df_test_null.index, y = df_test_null['missing'],
               name='df_test',text=df_test_null.index)

data = [trace1,trace2]

layout = dict(title='NaN in test and train',
             xaxis=dict(ticklen=10,zeroline=False),
             yaxis=dict(title='number of row',side='left',ticklen=10),
             legend=dict(orientation='v',x=1.05,y=1.0),
             autosize=False,width=750,height=500,
             barmode='stack')

fig = dict(data=data,layout=layout)
iplot(fig)

In [37]:
'''
nanデータが多いcolumnsがあればdrop
'''

'\nnanデータが多いcolumnsがあればdrop\n'

In [38]:
numerical_columns = df_train.select_dtypes(exclude=['object']).columns.tolist()
print(numerical_columns)

['Order', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area', 'BsmtFin SF 1', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold', 'SalePrice']


In [40]:
df_train['SalePrice_Log'] = np.log1p(df_train['SalePrice'])

In [43]:
fig = tools.make_subplots(rows=1,cols=2,print_grid=False,
                         subplot_titles=['SalePrice','SalePriceLog'])

trace_1 = go.Histogram(x=df_train['SalePrice'],name='SalePrice')
trace_2 = go.Histogram(x=df_train['SalePrice_Log'],name='SalePriceLog')

fig.append_trace(trace_1,1,1)
fig.append_trace(trace_2,1,2)

iplot(fig)

In [44]:
from scipy.stats import skew,kurtosis
print(df_train['SalePrice'].skew(),' ',df_train['SalePrice'].kurtosis())
print(df_train['SalePrice_Log'].skew(), ' ',df_train['SalePrice_Log'].kurtosis())

1.2471508580161197   2.589674073331626
-7.068831421095014   69.22868812293132


In [51]:
df_corr = df_train.corrwith(df_train['SalePrice']).abs().sort_values(ascending=False)[2:]

data = go.Bar(x=df_corr.index,
             y=df_corr.values)

layout = go.Layout(title = 'Correlation to Sale Price',
                  xaxis = dict(title=''),
                  yaxis = dict(title='correlation'),
                  autosize=False,width=750,height=500)

fig = dict(data=[data],layout=layout)
iplot(fig)

In [52]:
plotly_scatter_x_y(df_train,'Gr Liv Area','SalePrice')

In [53]:
outliers_GrLivArea = df_train.loc[(df_train['Gr Liv Area']<500.0) & (df_train['SalePrice']>400000.0)]
outliers_GrLivArea[['Gr Liv Area','SalePrice']]

,Gr Liv Area,SalePrice
788,482,498569
1173,428,476567


In [61]:
df_train['sum_1SF_2SF_LowQualSF'] = df_train['1st Flr SF'] + df_train['2nd Flr SF'] + df_train['Low Qual Fin SF']
df_test['sum_1SF_2SF_LowQualSF'] = df_test['1st Flr SF'] + df_test['2nd Flr SF'] + df_test['Low Qual Fin SF']

print(sum(df_train['sum_1SF_2SF_LowQualSF']!=df_train['Gr Liv Area']))
print(sum(df_test['sum_1SF_2SF_LowQualSF']!=df_test['Gr Liv Area']))

1199
800


In [62]:
df_train['sum_1SF_2SF_LowQualSF'].corr(df_train['SalePrice'])

-0.03495964649749097

In [63]:
(df_train['Gr Liv Area'] - df_train['Low Qual Fin SF']).corr(df_train['SalePrice'])

-0.09698625354050425

In [64]:
df_train.drop('sum_1SF_2SF_LowQualSF',axis=1,inplace=True)
df_test.drop('sum_1SF_2SF_LowQualSF',axis=1,inplace=True)

In [68]:
y_cols_val = 'SalePrice'

area_features = ['Total Bsmt SF','1st Flr SF','2nd Flr SF',
             'Mas Vnr Area','Garage Area','Lot Area',
             'Wood Deck SF','Open Porch SF','BsmtFin SF 1']
x_cols_val = area_features

In [70]:
nr_rows=3
nr_cols=3

fig = tools.make_subplots(rows=nr_rows,cols=nr_cols,print_grid=False,
                         subplot_titles=area_features)

for row in range(1,nr_rows+1):
    for col in range(1,nr_cols+1):
        i = (row-1) * nr_cols + col-1
        
        trace = go.Scatter(x = df_train[x_cols_val[i]],
                          y = df_train[y_cols_val],
                          name=x_cols_val[i],
                          mode='markers',
                          opacity=0.8)
        
        fig.append_trace(trace,row,col)
        
fig['layout'].update(height=700,width=900,showlegend=False,
                            title='SalePrice' + 'vs. Area features')
        
iplot(fig)

In [71]:
df_train['all_Liv_SF'] = df_train['Total Bsmt SF'] + df_train['1st Flr SF'] + df_train['2nd Flr SF']
df_test['all_Liv_SF'] = df_test['Total Bsmt SF'] + df_test['1st Flr SF'] + df_test['2nd Flr SF']

print(df_train['all_Liv_SF'].corr(df_train['SalePrice']))
print(df_train['all_Liv_SF'].corr(df_train['SalePrice_Log']))

-0.09044272788894167
-0.027230739941953956


In [72]:
df_train['all_SF'] = df_train['all_Liv_SF'] + df_train['Garage Area'] + df_train['Mas Vnr Area'] + df_train['Wood Deck SF'] + df_train['Open Porch SF'] + df_train['Screen Porch']
df_test['all_SF'] = df_test['all_Liv_SF'] + df_test['Garage Area'] + df_test['Mas Vnr Area'] + df_test['Wood Deck SF'] + df_test['Open Porch SF'] + df_test['Screen Porch']

print(df_train['all_SF'].corr(df_train['SalePrice']))
print(df_train['all_SF'].corr(df_train['SalePrice_Log']))

-0.08008145641737678
-0.016754983302587047


In [73]:
df_train['all_SF'].corr(df_train['all_Liv_SF'])

0.8953729554525677

In [74]:
plotly_scatter_x_y(df_train,'all_SF','SalePrice')

In [75]:
'''
本来であれば外れ値の除去を行う
'''

'\n本来であれば外れ値の除去を行う\n'

In [76]:
df_train.corr().abs()[['SalePrice','SalePrice_Log']].sort_values(by='SalePrice',ascending=False)[2:16]

,SalePrice,SalePrice_Log
Full Bath,0.097644,0.074900
Gr Liv Area,0.097349,0.044020
Kitchen AbvGr,0.093685,0.058671
Total Bsmt SF,0.090773,0.048301
all_Liv_SF,0.090443,0.027231
all_SF,0.080081,0.016755
Yr Sold,0.069227,0.045166
Bsmt Full Bath,0.068896,0.026192
Year Remod/Add,0.060624,0.019644
1st Flr SF,0.059102,0.004879


In [78]:
trace = []

for name,group in df_train[['SalePrice','Overall Qual']].groupby('Overall Qual'):
    trace.append(go.Box(y=group['SalePrice'].values,name=name))
    
layout = go.Layout(title='Overall Qual',
                  xaxis=dict(title='Over Qual',ticklen=5,zeroline=False),
                  yaxis=dict(title='SalePrice',side='left'),
                  autosize=False,width=750,height=500)

fig = go.Figure(data=trace,layout=layout)

iplot(fig)

In [83]:
outliners_OverallQual_3 = df_train.loc[(df_train['Overall Qual']==3) & (df_train['SalePrice']>250000.0)]
outliners_OverallQual_4 = df_train.loc[(df_train['Overall Qual']==4) & (df_train['SalePrice']>250000.0)]
outliners_OverallQual_5 = df_train.loc[(df_train['Overall Qual']==5) & (df_train['SalePrice']>280000.0)]
outliners_OverallQual_6 = df_train.loc[(df_train['Overall Qual']==6) & (df_train['SalePrice']>300000.0)]
outliners_OverallQual_7 = df_train.loc[(df_train['Overall Qual']==7) & (df_train['SalePrice']>280000.0)]
outliners_OverallQual_8 = df_train.loc[(df_train['Overall Qual']==8) & (df_train['SalePrice']>207500.0)]
outliners_OverallQual_9 = df_train.loc[(df_train['Overall Qual']==9) & (df_train['SalePrice']>300000.0)]
outliners_OverallQual_10 = df_train.loc[(df_train['Overall Qual']==10) & (df_train['SalePrice']>300000.0)]


In [86]:
outliners_OverallQual = pd.concat([outliners_OverallQual_3,outliners_OverallQual_4,
                                  outliners_OverallQual_5,outliners_OverallQual_6,
                                  outliners_OverallQual_7,outliners_OverallQual_8,
                                  outliners_OverallQual_9,outliners_OverallQual_10])

In [87]:
outliners_OverallQual[['Overall Qual','SalePrice']]

,Overall Qual,SalePrice
411,3,253288
60,4,308933
193,4,365007
434,4,381465
473,4,301123
...,...,...
1189,8,213039
1199,8,210504
603,9,340208
786,9,377870


In [88]:
df_train = df_train.drop(outliners_OverallQual.index)

In [89]:
df_train.corr().abs()[['SalePrice','SalePrice_Log']].sort_values(by='SalePrice',ascending=False)[2:20]

,SalePrice,SalePrice_Log
all_Liv_SF,0.123561,0.028056
Total Bsmt SF,0.110582,0.045646
all_SF,0.083710,0.010510
Kitchen AbvGr,0.081689,0.047150
Gr Liv Area,0.073108,0.029496
Bsmt Full Bath,0.068593,0.020312
Yr Sold,0.068043,0.038087
Full Bath,0.060641,0.059004
3Ssn Porch,0.057174,0.020021
Garage Area,0.055737,0.038345


In [97]:
plotly_scatter_x_y_catg_color(df_train,'all_SF','SalePrice','Overall Qual')

In [104]:
plotly_scatter3d(df_train,'all_SF','Overall Qual','SalePrice')

In [105]:
print(df_train['Overall Qual'].corr(df_train['all_SF']))

-0.043576894196822855


In [106]:
print(df_train['Overall Cond'].corr(df_train['SalePrice']))
print(df_train['Overall Cond'].corr(df_train['SalePrice_Log']))

-0.020458156561007877
-0.0014657949054501008


In [107]:
print(df_train['MS SubClass'].corr(df_train['SalePrice']))
print(df_train['MS SubClass'].corr(df_train['SalePrice_Log']))

-0.003979593494493825
0.020554848685425016


In [108]:
categorical_columns = df_train.select_dtypes(include=['object']).columns.tolist()

In [114]:
def plotly_boxplots_sorted_by_yvals(df,catg_feature,sort_by_target):
    
    df_by_catg = df.groupby([catg_feature])
    sortedlist_catg_str = df_by_catg[sort_by_target].median().sort_values().keys().tolist()
    
    data = []
    for i in sortedlist_catg_str:
        data.append(go.Box(y=df[df[catg_feature]==i][sort_by_target],name=i))
        
    layout = go.Layout(title=sort_by_target + 'vs ' + catg_feature,
                      xaxis=dict(title=catg_feature),
                      yaxis=dict(title=sort_by_target))
    
    fig = dict(data=data,layout=layout)
    return fig

In [115]:
fig = plotly_boxplots_sorted_by_yvals(df_train,'Neighborhood','SalePrice')
iplot(fig)

In [116]:
fig = plotly_boxplots_sorted_by_yvals(df_train,'MS Zoning','SalePrice')
iplot(fig)

In [117]:
outliners_all = []
df_train = df_train.drop(outliners_all)

In [118]:
y, y_log = df_train['SalePrice'], df_train['SalePrice_Log']
df_train.drop(['SalePrice','SalePrice_Log'],axis=1,inplace=True)

In [119]:
X_1 = df_train
y_1 = y_log

In [120]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [122]:
numerical_features = df_train.select_dtypes(exclude=['object']).columns.tolist()
categorical_features = df_train.select_dtypes(include=['object']).columns.tolist()

In [124]:
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

In [126]:
categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

In [127]:
preprocessor = ColumnTransformer(transformers=[
    ('num',numerical_transformer,numerical_features),
    ('cat',categorical_transformer,categorical_features)
])

In [129]:
pipe_Linear = Pipeline(
    steps = [('preprocessor',preprocessor),
           ('Linear',LinearRegression())])

pipe_Ridge = Pipeline(
    steps = [('preprocessor',preprocessor),
           ('Ridge',Ridge(random_state=2021))])

pipe_Huber = Pipeline(
    steps = [('preprocessor',preprocessor),
           ('Huber',HuberRegressor())])

pipe_Lasso = Pipeline(
    steps = [('preprocessor',preprocessor),
            ('Lasso',Lasso(random_state=2021))])

pipe_BayesianRidge = Pipeline(
    steps = [('preprocessor',preprocessor),
           ('BayesRidge',BayesianRidge(n_iter=500,compute_score=True))])

pipe_ElaNet = Pipeline(
    steps = [('preprocessor',preprocessor),
           ('ElaNet',ElasticNet(random_state=2021))])

In [130]:
#GradientBoostingRegressor
pipe_GBR = Pipeline(
    steps = [('preprocessor',preprocessor),
            ('GBR',GradientBoostingRegressor(random_state=2021))])

#XGBRegressor
pipe_XGB = Pipeline(
    steps = [('preprocessor',preprocessor),
            ('XGB',XGBRegressor(objective='reg:squarederror',metric='rmse',random_state=2021,nthread=-1))])

#LGBM
pipe_LGBM = Pipeline(
    steps = [('preprocessor',preprocessor),
            ('LGBM',LGBMRegressor(objective='regression',metric='rmse',random_state=2021))])

#AdaBoostRegressor
pipe_ADA = Pipeline(
    steps = [('preprocessor',preprocessor),
            ('ADA',AdaBoostRegressor(DecisionTreeRegressor(),
                                    random_state=2021,loss='exponential'))])

In [135]:
list_pipelines = [pipe_Linear,pipe_Ridge,pipe_Huber,pipe_Lasso,pipe_ElaNet]

In [136]:
print('model','\t','mean rmse','\t','std','\t','\t','min rmse')
print('*'*60)

for pipe in list_pipelines:
    
    scores = cross_val_score(pipe,X_1,y_1,scoring='neg_mean_squared_error',cv=5)
    scores = np.sqrt(-scores)
    print(pipe.steps[1][0],'\t',
         '{:08.6f}'.format(np.mean(scores)),'\t',
         '{:08.6f}'.format(np.std(scores)),'\t',
         '{:08.6f}'.format(np.min(scores)))

model 	 mean rmse 	 std 	 	 min rmse
****************************************
Linear 	 1.302851 	 0.247913 	 0.968666
Ridge 	 1.231430 	 0.269549 	 0.868829


C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/

Huber 	 1.092326 	 0.334037 	 0.630033
Lasso 	 1.046605 	 0.344312 	 0.607069
ElaNet 	 1.046605 	 0.344312 	 0.607069


In [137]:
list_pipelines = [pipe_GBR,pipe_XGB,pipe_LGBM,pipe_ADA]

In [142]:
print('model','\t','mean rmse','\t','std','\t','\t','min rmse')
print('+'*60)

for pipe in list_pipelines:
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        scores = cross_val_score(pipe,X_1,y_1,scoring='neg_mean_squared_error',cv=5)
        scores = np.sqrt(-scores)
        print(pipe.steps[1][0],'\t',
             '{:08.6f}'.format(np.mean(scores)),'\t',
             '{:08.6f}'.format(np.std(scores)),'\t',
             '{:08.6f}'.format(np.min(scores)))

model 	 mean rmse 	 std 	 	 min rmse
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
GBR 	 1.199331 	 0.304146 	 0.877711
[14:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:10:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:5

In [143]:
list_scalers = [StandardScaler(),
               RobustScaler(),
               QuantileTransformer(output_distribution='normal')]

In [151]:
list_scalers = [StandardScaler()]

In [154]:

parameters_Linear = {'preprocessor__num__scaler': list_scalers,
                     'Linear__fit_intercept': [True,False],
                     'Linear__normalize':[True,False]
                    }

gscv_Linear = GridSearchCV(pipe_Linear,parameters_Linear,n_jobs=-1,
                           scoring='neg_mean_squared_error',verbose=0,cv=5)

gscv_Linear.fit(X_1,y_1)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [155]:
print(np.sqrt(-gscv_Linear.best_score_))
gscv_Linear.best_params_

1.3258827220635607


{'Linear__fit_intercept': False,
 'Linear__normalize': True,
 'preprocessor__num__scaler': StandardScaler()}

In [158]:
parameters_Ridge = {
    'preprocessor__num__scaler':list_scalers,
    'Ridge__alpha':[7,8,9],
    'Ridge__fit_intercept':[True,False],
    'Ridge__normalize':[True,False]
}

gscv_Ridge = GridSearchCV(pipe_Ridge,parameters_Ridge,n_jobs=-1,
                         scoring='neg_mean_squared_error',verbose=0,cv=5)

gscv_Ridge.fit(X_1,y_1)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [159]:
print(np.sqrt(-gscv_Ridge.best_score_))
gscv_Ridge.best_params_

1.1009260210765741


{'Ridge__alpha': 9,
 'Ridge__fit_intercept': True,
 'Ridge__normalize': True,
 'preprocessor__num__scaler': StandardScaler()}

In [161]:
parameters_Huber = {
    'preprocessor__num__scaler':list_scalers,
    'Huber__epsilon':[1.3,1.35,1.4],
    'Huber__max_iter':[150,200,250],
    'Huber__alpha':[0.0005,0.001,0.002],
    'Huber__fit_intercept':[True]
}

gscv_Huber = GridSearchCV(pipe_Huber,parameters_Huber,n_jobs=-1,
                         scoring='neg_mean_squared_error',verbose=0,cv=5)

gscv_Huber.fit(X_1,y_1)

C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [162]:
print(np.sqrt(-gscv_Huber.best_score_))
gscv_Huber.best_params_

1.148947069565117


{'Huber__alpha': 0.0005,
 'Huber__epsilon': 1.4,
 'Huber__fit_intercept': True,
 'Huber__max_iter': 150,
 'preprocessor__num__scaler': StandardScaler()}

In [165]:
parameters_Lasso = {
    'preprocessor__num__scaler':list_scalers,
    'Lasso__alpha':[0.0005,0.001],
    'Lasso__fit_intercept':[True],
    'Lasso__normalize':[True,False]
}

gscv_Lasso = GridSearchCV(pipe_Lasso,parameters_Lasso,n_jobs=-1,
                         scoring='neg_mean_squared_error',verbose=1,cv=5)

gscv_Lasso.fit(X_1,y_1)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [166]:
print(np.sqrt(-gscv_Lasso.best_score_))
gscv_Lasso.best_params_

1.170879654787291


{'Lasso__alpha': 0.001,
 'Lasso__fit_intercept': True,
 'Lasso__normalize': True,
 'preprocessor__num__scaler': StandardScaler()}

In [167]:
parameters_ElaNet = {
    'ElaNet__alpha':[0.0005,0.001],
    'ElaNet__l1_ratio':[0.85,0.9],
    'ElaNet__normalize':[True,False]
}

gscv_ElaNet = GridSearchCV(pipe_ElaNet,parameters_ElaNet,n_jobs=-1,
                          scoring='neg_mean_squared_error',verbose=1,cv=5)

gscv_ElaNet.fit(X_1,y_1)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [180]:
print(np.sqrt(-gscv_ElaNet.best_score_))
gscv_ElaNet.best_params_

1.1598750879022384


{'ElaNet__alpha': 0.001, 'ElaNet__l1_ratio': 0.85, 'ElaNet__normalize': True}

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [173]:
list_pipelines_gscv = [gscv_Linear,gscv_Ridge,gscv_Huber,gscv_Lasso,gscv_ElaNet]

In [176]:
print('model','\t','mean rmse','\t','std','\t','\t','min rmse')
print('+'*60)

for gscv in list_pipelines_gscv:
    scores = cross_val_score(gscv.best_estimator_,X_1,y_1,scoring='neg_mean_squared_error',cv=5)
    scores = np.sqrt(-scores)
    print(gscv.estimator.steps[1][0],'\t',
        '{:08.6f}'.format(np.mean(scores)),'\t',
        '{:08.6f}'.format(np.std(scores)),'\t',
        '{:08.6f}'.format(np.min(scores)))

model 	 mean rmse 	 std 	 	 min rmse
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Linear 	 1.302475 	 0.248039 	 0.967594
Ridge 	 1.046327 	 0.342400 	 0.603493


C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

C:\Users\81806\anaconda3\envs\kaggle_book\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/

Huber 	 1.100197 	 0.331129 	 0.638793
Lasso 	 1.122951 	 0.331571 	 0.704314
ElaNet 	 1.111925 	 0.330051 	 0.692227


In [182]:
parameters_GBR = {
    'GBR__n_estimators':[400],
    'GBR__max_depth':[3,4,5,6,7,8],
    'GBR__min_samples_leaf':[5,6],
    'GBR__max_features':['auto',0.5,0.7]
}

gscv_GBR = GridSearchCV(pipe_GBR,parameters_GBR,n_jobs=-1,
                       scoring='neg_mean_squared_error',verbose=1,cv=5)

gscv_GBR.fit(X_1,y_1)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [183]:
print(np.sqrt(-gscv_GBR.best_score_))
gscv_GBR.best_params_

1.1781894783116433


{'GBR__max_depth': 8,
 'GBR__max_features': 0.7,
 'GBR__min_samples_leaf': 5,
 'GBR__n_estimators': 400}

In [185]:
parameters_XGB = {
    'XGB__learning_rate':[0.021,0.022],
    'XGB__max_depth':[2,3],
    'XGB__n_estimators': [2000],
    'XGB__reg_lambda':[1.5,1.6],
    'XGB__reg_alpha':[1,1.5]
}

gscv_XGB = GridSearchCV(pipe_XGB,parameters_XGB,n_jobs=-1,
                       scoring='neg_mean_squared_error',verbose=1,cv=5)

gscv_XGB.fit(X_1,y_1)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[15:38:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [186]:
print(np.sqrt(-gscv_XGB.best_score_))
gscv_XGB.best_params_

1.213683542999063


{'XGB__learning_rate': 0.021,
 'XGB__max_depth': 2,
 'XGB__n_estimators': 2000,
 'XGB__reg_alpha': 1.5,
 'XGB__reg_lambda': 1.6}

In [187]:
parameters_LGBM = {
    'LGBM__learning_rate':[0.01,0.02],
    'LGBM__n_estimators':[1000],
    'LGBM__num_leaves':[8,10],
    'LGBM__bagging_fraction':[0.7,0.8],
    'LGBM__gagging_freq':[1,2]
}

gscv_LGBM = GridSearchCV(pipe_LGBM,parameters_LGBM,n_jobs=-1,
                        scoring='neg_mean_squared_error',verbose=1,cv=5)

gscv_LGBM.fit(X_1,y_1)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[LightGBM] [Warning] Unknown parameter: gagging_freq
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [188]:
print(np.sqrt(-gscv_LGBM.best_score_))
gscv_LGBM.best_params_

1.1410626191718052


{'LGBM__bagging_fraction': 0.7,
 'LGBM__gagging_freq': 1,
 'LGBM__learning_rate': 0.01,
 'LGBM__n_estimators': 1000,
 'LGBM__num_leaves': 8}

In [191]:
parameters_ADA = {
    'ADA__learning_rate':[3.5],
    'ADA__n_estimators':[500],
    'ADA__base_estimator__max_depth':[8,9,10]
}

pipe_ADA = Pipeline(
    steps = [('preprocessor',preprocessor),
            ('ADA',AdaBoostRegressor(DecisionTreeRegressor(min_samples_leaf=5,min_samples_split=5),
                                    random_state=2021,loss='exponential'))])

gscv_ADA = GridSearchCV(pipe_ADA,parameters_ADA,n_jobs=-1,
                       scoring='neg_mean_squared_error',verbose=1,cv=5)

gscv_ADA.fit(X_1,y_1)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Order',
                                                                          'PID',
                                                                          'MS '
                                                                          'SubClass',
                                                                          'Lot

In [192]:
print(np.sqrt(-gscv_ADA.best_score_))
gscv_ADA.best_params_

1.576963612227453


{'ADA__base_estimator__max_depth': 9,
 'ADA__learning_rate': 3.5,
 'ADA__n_estimators': 500}

In [193]:
list_pipelines_gscv = [gscv_GBR,gscv_XGB,gscv_LGBM,gscv_ADA]

In [195]:
print('model','\t','mean rmse','\t','std','\t','\t','min rmse')
print('+'*60)

for gscv in list_pipelines_gscv:
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        scores = cross_val_score(gscv.best_estimator_,X_1,y_1,scoring='neg_mean_squared_error',cv=5)
        scores = np.sqrt(-scores)
        print(gscv.estimator.steps[1][0],'\t',
             '{:08.6f}'.format(np.mean(scores)),'\t',
             '{:08.6f}'.format(np.std(scores)),'\t',
             '{:08.6f}'.format(np.min(scores)))

model 	 mean rmse 	 std 	 	 min rmse
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
GBR 	 1.137097 	 0.308450 	 0.731720
[16:16:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:16:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:16:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:5

In [196]:
linear_models = [gscv_Linear,gscv_Ridge,gscv_Huber,gscv_Lasso,gscv_ElaNet]
boost_models = [gscv_GBR,gscv_XGB,gscv_LGBM,gscv_ADA]

In [198]:
pred_Linear = gscv_Linear.predict(df_test)
pred_Ridge = gscv_Ridge.predict(df_test)
pred_Huber = gscv_Huber.predict(df_test)
pred_Lasso = gscv_Lasso.predict(df_test)
pred_ElaNet = gscv_ElaNet.predict(df_test)

In [199]:
prediction_Linear = {'Linear':pred_Linear,'Ridge':pred_Ridge,'Huber':pred_Huber,
                    'Lasso':pred_Lasso,'ElaNet':pred_ElaNet}

In [201]:
output = Path('output')

for model,values in prediction_Linear.items():
    str_filename = model + '.csv'
    print('writing submission to : ',str_filename)
    subm = pd.DataFrame()
    subm['index'] = id_test
    subm['SalePrice'] = np.expm1(values)
    subm.to_csv(output / str_filename,index=False,header=None)

writing submission to :  Linear.csv
writing submission to :  Ridge.csv
writing submission to :  Huber.csv
writing submission to :  Lasso.csv
writing submission to :  ElaNet.csv


In [202]:
pred_Blend_1 = (pred_Lasso + pred_Ridge) / 2
sub_Blend_1 = pd.DataFrame()
sub_Blend_1['index'] = id_test
sub_Blend_1['SalePrice'] = np.expm1(pred_Blend_1)
sub_Blend_1.to_csv(output / 'Blend_Ridge_Lasso.csv',index=False,header=None)
sub_Blend_1.head()

,index,SalePrice
0,1,86050.543419
1,2,95802.884827
2,4,106062.296597
3,5,105990.074490
4,6,98846.831907


In [204]:
pred_Blend_2 = (pred_Lasso + pred_ElaNet) / 2
sub_Blend_2 = pd.DataFrame()
sub_Blend_2['index'] = id_test
sub_Blend_2['SalePrice'] = np.expm1(pred_Blend_2)
sub_Blend_2.to_csv(output / 'Blend_Lasso_ElaNet.csv',index=False,header=None)
sub_Blend_2.head()

,index,SalePrice
0,1,79011.018125
1,2,93105.008679
2,4,113780.753323
3,5,114245.462587
4,6,96343.727806


In [206]:
pred_Blend_3 = (pred_Ridge + pred_Lasso + pred_ElaNet) / 3
sub_Blend_3 = pd.DataFrame()
sub_Blend_3['index'] = id_test
sub_Blend_3['SalePrice'] = np.expm1(pred_Blend_3)
sub_Blend_3.to_csv(output / 'Blend_Ridge_Lasso_ElaNet.csv',index=False,header=None)
sub_Blend_3.head()

,index,SalePrice
0,1,83517.299617
1,2,94325.976510
2,4,109165.816278
3,5,108678.144102
4,6,98137.718384


In [207]:
boost_models = [gscv_GBR,gscv_XGB,gscv_LGBM,gscv_ADA]

In [208]:
pred_GBR = gscv_GBR.predict(df_test)
pred_XGB = gscv_XGB.predict(df_test)
pred_LGBM = gscv_LGBM.predict(df_test)
pred_ADA = gscv_ADA.predict(df_test)

In [209]:
prediction_booster = {'GBR':pred_GBR,'XGB':pred_XGB,'LGBM':pred_LGBM,'ADA':pred_ADA}

In [210]:
for model,values in prediction_booster.items():
    str_filename = model + '.csv'
    print('witing submission to : ',str_filename)
    subm = pd.DataFrame()
    subm['index'] = id_test
    subm['SalePrice'] = np.expm1(values)
    subm.to_csv(output / str_filename,index=False,header=None)

witing submission to :  GBR.csv
witing submission to :  XGB.csv
witing submission to :  LGBM.csv
witing submission to :  ADA.csv


In [211]:
predictions = {
    'Ridge':pred_Ridge,'Lasso':pred_Lasso,'ElaNet':pred_ElaNet,
    'GBR':pred_GBR,'XGB':pred_XGB,'LGBM':pred_LGBM,'ADA':pred_ADA
             }
df_predictions = pd.DataFrame(data=predictions)
df_predictions.corr()

,Ridge,Lasso,ElaNet,GBR,XGB,LGBM,ADA
Ridge,1.000000,0.788092,0.828855,0.381059,0.446269,0.384874,0.331334
Lasso,0.788092,1.000000,0.994963,0.361460,0.516969,0.256875,0.230616
ElaNet,0.828855,0.994963,1.000000,0.377009,0.522180,0.290050,0.260262
GBR,0.381059,0.361460,0.377009,1.000000,0.645260,0.465530,0.360459
XGB,0.446269,0.516969,0.522180,0.645260,1.000000,0.371536,0.301107
LGBM,0.384874,0.256875,0.290050,0.465530,0.371536,1.000000,0.553710
ADA,0.331334,0.230616,0.260262,0.360459,0.301107,0.553710,1.000000


In [212]:
pred_Blend_10 = (pred_Ridge + pred_XGB) / 2
sub_Blend_10 = pd.DataFrame()
sub_Blend_10['index'] = id_test
sub_Blend_10['SalePrice'] = np.expm1(pred_Blend_10)
sub_Blend_10.to_csv(output/'Blend_Ridge_XGB',index=False,header=None)
sub_Blend_10.head()

,index,SalePrice
0,1,66620.181162
1,2,77900.005794
2,4,70924.376558
3,5,104333.170004
4,6,104390.611271


In [214]:
lnr = LinearRegression(n_jobs=-1)

rdg = Ridge(alpha=3.0,copy_X=True,fit_intercept=True,random_state=2021)

rft = RandomForestRegressor(n_estimators=12,max_depth=3,n_jobs=-1,random_state=2021)

gbr = GradientBoostingRegressor(n_estimators=40,max_depth=2,random_state=2021)

mlp = MLPRegressor(hidden_layer_sizes=(90,90),alpha=2.75,random_state=2021)

In [215]:
stack1 = StackingRegressor(regressors=[rdg,rft,gbr],meta_regressor=lnr)

In [216]:
pipe_STACK_1 = Pipeline(
    steps=[('preprocessor',preprocessor),
          ('stack1',stack1)])

pipe_STACK_1.fit(X_1,y_1)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Order', 'PID',
                                                   'MS SubClass',
                                                   'Lot Frontage', 'Lot Area',
                                                   'Overall Qual',
                                                   'Overall Cond', 'Year Built',
                                                   'Year Remod/Add',
                                                   'Mas Vnr Area',
                                                   'BsmtFin SF 1',
  

In [217]:
pred_stack1 = pipe_STACK_1.predict(df_test)
sub_stack1 = pd.DataFrame()
sub_stack1['index'] = id_test
sub_stack1['SalePrice'] = np.expm1(pred_stack1)
sub_stack1.to_csv(output/'pipe_stack1.csv',index=False,header=None)

In [218]:
sub_stack1.head(10)

,index,SalePrice
0,1,57262.372561
1,2,104262.575327
2,4,88324.823162
3,5,115550.590851
4,6,118933.037899
5,8,132193.332078
6,9,139335.152044
7,13,112135.285760
8,14,84865.136466
9,16,130863.086425
